<a href="https://colab.research.google.com/github/Sarvjeet-Srivastava/probability-distribution/blob/master/even-easier-cuda/An_Even_Easier_Introduction_to_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Even Easier Introduction to CUDA

This notebook accompanies Mark Harris's popular blog post [_An Even Easier Introduction to CUDA_](https://developer.nvidia.com/blog/even-easier-introduction-cuda/).

If you enjoy this notebook and want to learn more, the [NVIDIA DLI](https://nvidia.com/dli) offers several in depth CUDA Programming courses.

For those of you just starting out, please consider [_Fundamentals of Accelerated Computing with CUDA C/C++_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-01+V1/about) which provides dedicated GPU resources, a more sophisticated programming environment, use of the [NVIDIA Nsight Systems™](https://developer.nvidia.com/nsight-systems) visual profiler, dozens of interactive exercises, detailed presentations, over 8 hours of material, and the ability to earn a DLI Certificate of Competency.

Similarly, for Python programmers, please consider [_Fundamentals of Accelerated Computing with CUDA Python_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-02+V1/about).

For more intermediate and advance CUDA programming materials, please check out the _Accelerated Computing_ section of the NVIDIA DLI [self-paced catalog](https://www.nvidia.com/en-us/training/online/).

<img src="https://developer.download.nvidia.com/training/courses/T-AC-01-V1/CUDA_Cube_1K.jpeg" width="400">

This post is a super simple introduction to CUDA, the popular parallel computing platform and programming model from NVIDIA. I wrote a previous [“Easy Introduction”](https://developer.nvidia.com/blog/easy-introduction-cuda-c-and-c/) to CUDA in 2013 that has been very popular over the years. But CUDA programming has gotten easier, and GPUs have gotten much faster, so it’s time for an updated (and even easier) introduction.

CUDA C++ is just one of the ways you can create massively parallel applications with CUDA. It lets you use the powerful C++ programming language to develop high performance algorithms accelerated by thousands of parallel threads running on GPUs. Many developers have accelerated their computation- and bandwidth-hungry applications this way, including the libraries and frameworks that underpin the ongoing revolution in artificial intelligence known as [Deep Learning](https://developer.nvidia.com/deep-learning).

So, you’ve heard about CUDA and you are interested in learning how to use it in your own applications. If you are a C or C++ programmer, this blog post should give you a good start. To follow along, you’ll need a computer with an CUDA-capable GPU (Windows, Mac, or Linux, and any NVIDIA GPU should do), or a cloud instance with GPUs (AWS, Azure, IBM SoftLayer, and other cloud service providers have them). You’ll also need the free [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit) installed.

Let's get started!

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_ai_cube-625x625.jpg" width="400">

## Starting Simple

We’ll start with a simple C++ program that adds the elements of two arrays with a million elements each.

In [ ]:
%%writefile add.cpp

#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}

Executing the above cell will save its contents to the file add.cpp.

The following cell will compile and run this C++ program.

In [ ]:
%%shell
g++ add.cpp -o add

Then run it:

In [ ]:
%%shell
./add

As expected, it prints that there was no error in the summation and then exits. Now I want to get this computation running (in parallel) on the many cores of a GPU. It’s actually pretty easy to take the first steps.

First, I just have to turn our `add` function into a function that the GPU can run, called a *kernel* in CUDA. To do this, all I have to do is add the specifier `__global__` to the function, which tells the CUDA C++ compiler that this is a function that runs on the GPU and can be called from CPU code.

```cpp
// CUDA Kernel function to add the elements of two arrays on the GPU
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}
```

These `__global__` functions are known as *kernels*, and code that runs on the GPU is often called *device code*, while code that runs on the CPU is *host code*.

## Memory Allocation in CUDA

To compute on the GPU, I need to allocate memory accessible by the GPU. [Unified Memory](https://developer.nvidia.com/blog/unified-memory-in-cuda-6/) in CUDA makes this easy by providing a single memory space accessible by all GPUs and CPUs in your system. To allocate data in unified memory, call `cudaMallocManaged()`, which returns a pointer that you can access from host (CPU) code or device (GPU) code. To free the data, just pass the pointer to `cudaFree()`.

I just need to replace the calls to `new` in the code above with calls to `cudaMallocManaged()`, and replace calls to `delete []` with calls to `cudaFree`.

```cpp
  // Allocate Unified Memory -- accessible from CPU or GPU
  float *x, *y;
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  ...

  // Free memory
  cudaFree(x);
  cudaFree(y);
```

Finally, I need to *launch* the `add()` kernel, which invokes it on the GPU. CUDA kernel launches are specified using the triple angle bracket syntax `<<< >>>`. I just have to add it to the call to `add` before the parameter list.

```cpp
add<<<1, 1>>>(N, x, y);
```

Easy! I’ll get into the details of what goes inside the angle brackets soon; for now all you need to know is that this line launches one GPU thread to run `add()`.

Just one more thing: I need the CPU to wait until the kernel is done before it accesses the results (because CUDA kernel launches don’t block the calling CPU thread). To do this I just call `cudaDeviceSynchronize()` before doing the final error checking on the CPU.

Here’s the complete code:

In [35]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20
 ;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting add.cu


In [36]:
%%shell

nvcc add.cu -o add_cuda
./add_cuda

Max error: 1


This is only a first step, because as written, this kernel is only correct for a single thread, since every thread that runs it will perform the add on the whole array. Moreover, there is a [race condition](https://en.wikipedia.org/wiki/Race_condition) since multiple parallel threads would both read and write the same locations.

## Profile it!

I think the simplest way to find out how long the kernel takes to run is to run it with `nvprof`, the command line GPU profiler that comes with the CUDA Toolkit. Just type `nvprof ./add_cuda` on the command line:

In [37]:
%%shell

nvprof ./add_cuda

==12266== NVPROF is profiling process 12266, command: ./add_cuda
Max error: 1
==12266== Profiling application: ./add_cuda
==12266== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   96.34%  218.57ms         2  109.29ms  42.066us  218.53ms  cudaMallocManaged
                    3.43%  7.7772ms         1  7.7772ms  7.7772ms  7.7772ms  cudaLaunchKernel
                    0.13%  304.78us         2  152.39us  113.83us  190.95us  cudaFree
                    0.08%  189.81us       114  1.6640us     102ns  64.416us  cuDeviceGetAttribute
                    0.01%  14.443us         1  14.443us  14.443us  14.443us  cuDeviceGetName
                    0.01%  13.230us         1  13.230us  13.230us  13.230us  cudaDeviceSynchronize
                    0.00%  6.4390us         1  6.4390us  6.4390us  6.4390us  cuDeviceGetPCIBusId
                    0.00%  2.1090us         3     703ns     118ns  1.7580us  cuD

The above will show the single call to `add`. Your timing may vary depending on the GPU allocated to you by Colab. To see the current GPU allocated to you run the following cell and look in the `Name` column where you might see, for example `Tesla T4`:

In [38]:
%%shell

nvidia-smi

Thu Oct 16 20:26:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Let's make it faster with parallelism.

## Picking up the Threads

Now that you’ve run a kernel with one thread that does some computation, how do you make it parallel? The key is in CUDA’s `<<<1, 1>>>` syntax. This is called the execution configuration, and it tells the CUDA runtime how many parallel threads to use for the launch on the GPU. There are two parameters here, but let’s start by changing the second one: the number of threads in a thread block. CUDA GPUs run kernels using blocks of threads that are a multiple of 32 in size, so 256 threads is a reasonable size to choose.

```cpp
add<<<1, 256>>>(N, x, y);
```

If I run the code with only this change, it will do the computation once per thread, rather than spreading the computation across the parallel threads. To do it properly, I need to modify the kernel. CUDA C++ provides keywords that let kernels get the indices of the running threads. Specifically, `threadIdx.x` contains the index of the current thread within its block, and `blockDim.x` contains the number of threads in the block. I’ll just modify the loop to stride through the array with parallel threads.

```cpp
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}
```

The `add` function hasn’t changed that much. In fact, setting `index` to 0 and `stride` to 1 makes it semantically identical to the first version.

Here we save the file as add_block.cu and compile and run it in `nvprof` again.

In [39]:
%%writefile add_block.cu

#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 256>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting add_block.cu


In [40]:
%%shell

nvcc add_block.cu -o add_block
nvprof ./add_block

==12427== NVPROF is profiling process 12427, command: ./add_block
Max error: 1
==12427== Profiling application: ./add_block
==12427== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   96.38%  209.89ms         2  104.94ms  55.598us  209.83ms  cudaMallocManaged
                    3.38%  7.3642ms         1  7.3642ms  7.3642ms  7.3642ms  cudaLaunchKernel
                    0.15%  316.45us         2  158.23us  120.11us  196.34us  cudaFree
                    0.07%  150.83us       114  1.3230us     103ns  67.320us  cuDeviceGetAttribute
                    0.01%  15.691us         1  15.691us  15.691us  15.691us  cuDeviceGetName
                    0.01%  14.576us         1  14.576us  14.576us  14.576us  cudaDeviceSynchronize
                    0.00%  6.0720us         1  6.0720us  6.0720us  6.0720us  cuDeviceGetPCIBusId
                    0.00%  2.1290us         3     709ns     165ns  1.6920us  c

That’s a big speedup (compare the time for the `add` kernel by looking at the `GPU activities` field), but not surprising since I went from 1 thread to 256 threads. Let’s keep going to get even more performance.

## Out of the Blocks

CUDA GPUs have many parallel processors grouped into Streaming Multiprocessors, or SMs. Each SM can run multiple concurrent thread blocks. As an example, a Tesla P100 GPU based on the [Pascal GPU Architecture](https://developer.nvidia.com/blog/inside-pascal/) has 56 SMs, each capable of supporting up to 2048 active threads. To take full advantage of all these threads, I should launch the kernel with multiple thread blocks.

By now you may have guessed that the first parameter of the execution configuration specifies the number of thread blocks. Together, the blocks of parallel threads make up what is known as the *grid*. Since I have `N` elements to process, and 256 threads per block, I just need to calculate the number of blocks to get at least `N` threads. I simply divide `N` by the block size (being careful to round up in case `N` is not a multiple of `blockSize`).

```cpp
int blockSize = 256;
int numBlocks = (N + blockSize - 1) / blockSize;
add<<<numBlocks, blockSize>>>(N, x, y);
```

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_indexing.png" width="800">

I also need to update the kernel code to take into account the entire grid of thread blocks. CUDA provides `gridDim.x`, which contains the number of blocks in the grid, and `blockIdx.x`, which contains the index of the current thread block in the grid. Figure 1 illustrates the the approach to indexing into an array (one-dimensional) in CUDA using `blockDim.x`, `gridDim.x`, and `threadIdx.x`. The idea is that each thread gets its index by computing the offset to the beginning of its block (the block index times the block size: `blockIdx.x * blockDim.x`) and adding the thread’s index within the block (`threadIdx.x`). The code `blockIdx.x * blockDim.x + threadIdx.x` is idiomatic CUDA.

```cpp
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}
```

The updated kernel also sets stride to the total number of threads in the grid (`blockDim.x * gridDim.x`). This type of loop in a CUDA kernel is often called a [*grid-stride*](https://developer.nvidia.com/blog/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/) loop.

Save the file as `add_grid.cu` and compile and run it in `nvprof` again.

In [5]:
%%writefile add_grid.cu

#include <iostream>
#include <math.h>
#include <stdio.h> // Include for printf
#include <cuda_runtime.h> // Include for CUDA error checking

// Macro for checking CUDA errors
#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while (0)

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  // Add printf for debugging (only for a few threads to avoid excessive output)
  if (index < 10 || index > n - 10) {
      printf("Thread %d: blockIdx.x=%d, threadIdx.x=%d, index=%d, n=%d, stride=%d\n",
             index, blockIdx.x, threadIdx.x, index, n, stride);
  }

  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  CUDA_CHECK(cudaMallocManaged(&x, N*sizeof(float)));
  CUDA_CHECK(cudaMallocManaged(&y, N*sizeof(float)));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(N, x, y);
  CUDA_CHECK(cudaGetLastError()); // Check for launch errors

  // Wait for GPU to finish before accessing on host
  CUDA_CHECK(cudaDeviceSynchronize());

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  CUDA_CHECK(cudaFree(x));
  CUDA_CHECK(cudaFree(y));

  return 0;
}

Overwriting add_grid.cu


In [7]:
%%shell

nvcc -arch=sm_75 add_grid.cu -o add_grid
./add_grid

Thread 0: blockIdx.x=0, threadIdx.x=0, index=0, n=1048576, stride=1048576
Thread 1: blockIdx.x=0, threadIdx.x=1, index=1, n=1048576, stride=1048576
Thread 2: blockIdx.x=0, threadIdx.x=2, index=2, n=1048576, stride=1048576
Thread 3: blockIdx.x=0, threadIdx.x=3, index=3, n=1048576, stride=1048576
Thread 4: blockIdx.x=0, threadIdx.x=4, index=4, n=1048576, stride=1048576
Thread 5: blockIdx.x=0, threadIdx.x=5, index=5, n=1048576, stride=1048576
Thread 6: blockIdx.x=0, threadIdx.x=6, index=6, n=1048576, stride=1048576
Thread 7: blockIdx.x=0, threadIdx.x=7, index=7, n=1048576, stride=1048576
Thread 8: blockIdx.x=0, threadIdx.x=8, index=8, n=1048576, stride=1048576
Thread 9: blockIdx.x=0, threadIdx.x=9, index=9, n=1048576, stride=1048576
Thread 1048567: blockIdx.x=4095, threadIdx.x=247, index=1048567, n=1048576, stride=1048576
Thread 1048568: blockIdx.x=4095, threadIdx.x=248, index=1048568, n=1048576, stride=1048576
Thread 1048569: blockIdx.x=4095, threadIdx.x=249, index=1048569, n=1048576, st

That's another big speedup from running multiple blocks! (Note your results may vary from the blog post due to whatever GPU you've been allocated by Colab. If you notice your speedups for the final example are not as drastic as those in the blog post, check out #4 in the *Exercises* section below.)

## Exercises

To keep you going, here are a few things to try on your own.

1. Browse the [CUDA Toolkit documentation](https://docs.nvidia.com/cuda/index.html). If you haven’t installed CUDA yet, check out the [Quick Start Guide](https://docs.nvidia.com/cuda/cuda-quick-start-guide/index.html) and the installation guides. Then browse the [Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html) and the [Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html). There are also tuning guides for various architectures.
2. Experiment with `printf()` inside the kernel. Try printing out the values of `threadIdx.x` and `blockIdx.x` for some or all of the threads. Do they print in sequential order? Why or why not?
3. Print the value of `threadIdx.y` or `threadIdx.z` (or `blockIdx.y`) in the kernel. (Likewise for `blockDim` and `gridDim`). Why do these exist? How do you get them to take on values other than 0 (1 for the dims)?
4. If you have access to a [Pascal-based GPU](https://developer.nvidia.com/blog/inside-pascal/), try running `add_grid.cu` on it. Is performance better or worse than the K80 results? Why? (Hint: read about [Pascal’s Page Migration Engine and the CUDA 8 Unified Memory API](https://developer.nvidia.com/blog/beyond-gpu-memory-limits-unified-memory-pascal/).) For a detailed answer to this question, see the post [Unified Memory for CUDA Beginners](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/).

In [8]:
%%writefile check_printf.cu

#include <iostream>
#include <math.h>
#include <stdio.h> // Include for printf

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  // Add printf for debugging (only for a few threads to avoid excessive output)
  if (index < 10 || index > n - 10) {
      printf("Thread %d: blockIdx.x=%d, threadIdx.x=%d, index=%d, n=%d, stride=%d\n",
             index, blockIdx.x, threadIdx.x, index, n, stride);
  }

  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing check_printf.cu


In [12]:
%%shell

nvcc -arch=sm_75 check_printf.cu -o check_printf
nvprof ./check_printf

==17089== NVPROF is profiling process 17089, command: ./check_printf
Thread 0: blockIdx.x=0, threadIdx.x=0, index=0, n=1048576, stride=1048576
Thread 1: blockIdx.x=0, threadIdx.x=1, index=1, n=1048576, stride=1048576
Thread 2: blockIdx.x=0, threadIdx.x=2, index=2, n=1048576, stride=1048576
Thread 3: blockIdx.x=0, threadIdx.x=3, index=3, n=1048576, stride=1048576
Thread 4: blockIdx.x=0, threadIdx.x=4, index=4, n=1048576, stride=1048576
Thread 5: blockIdx.x=0, threadIdx.x=5, index=5, n=1048576, stride=1048576
Thread 6: blockIdx.x=0, threadIdx.x=6, index=6, n=1048576, stride=1048576
Thread 7: blockIdx.x=0, threadIdx.x=7, index=7, n=1048576, stride=1048576
Thread 8: blockIdx.x=0, threadIdx.x=8, index=8, n=1048576, stride=1048576
Thread 9: blockIdx.x=0, threadIdx.x=9, index=9, n=1048576, stride=1048576
Thread 1048567: blockIdx.x=4095, threadIdx.x=247, index=1048567, n=1048576, stride=1048576
Thread 1048568: blockIdx.x=4095, threadIdx.x=248, index=1048568, n=1048576, stride=1048576
Thread 10

In [13]:
%%writefile check_printf2.cu

#include <iostream>
#include <math.h>
#include <stdio.h> // Include for printf
#include <cuda_runtime.h> // Include for CUDA error checking

// Macro for checking CUDA errors
#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while (0)

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  // Calculate 1D index from 2D block and thread indices
  int blockId = blockIdx.x + blockIdx.y * gridDim.x;
  int threadId = threadIdx.x + threadIdx.y * blockDim.x;
  int index = blockId * (blockDim.x * blockDim.y) + threadId;

  int stride = gridDim.x * gridDim.y * blockDim.x * blockDim.y;

  // Add printf for debugging (only for a few threads to avoid excessive output)
  if (index < 20 || index > n - 20) {
      printf("Thread (%d, %d): blockIdx=(%d, %d), threadIdx=(%d, %d), index=%d, n=%d, stride=%d, blockDim=(%d, %d), gridDim=(%d, %d)\n",
             threadIdx.x, threadIdx.y, blockIdx.x, blockIdx.y, threadIdx.x, threadIdx.y, index, n, stride, blockDim.x, blockDim.y, gridDim.x, gridDim.y);
  }

  // Ensure index is within bounds
  if (index < n) {
    y[index] = x[index] + y[index];
  }
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  CUDA_CHECK(cudaMallocManaged(&x, N*sizeof(float)));
  CUDA_CHECK(cudaMallocManaged(&y, N*sizeof(float)));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU with 2D grid and block
  int blockDimX = 16;
  int blockDimY = 16; // Using a 16x16 thread block (256 threads)
  dim3 blockSize(blockDimX, blockDimY);

  int gridDimX = (N + (blockDimX * blockDimY) - 1) / (blockDimX * blockDimY);
  int gridDimY = 1; // For simplicity, keep grid 1D in y for now

  // To demonstrate 2D grid, let's make gridDimX and gridDimY such that total blocks are sufficient
  int totalThreadsPerBlock = blockDimX * blockDimY;
  int totalBlocks = (N + totalThreadsPerBlock - 1) / totalThreadsPerBlock;
  gridDimX = 64; // Example: use a 64x64 grid of blocks
  gridDimY = 64;
  if (gridDimX * gridDimY < totalBlocks) {
      // Adjust grid dimensions if needed to cover all elements
      // A more robust calculation would be needed for a truly flexible 2D grid
      gridDimX = (totalBlocks + gridDimY - 1) / gridDimY;
  }
  dim3 numBlocks(gridDimX, gridDimY);


  add<<<numBlocks, blockSize>>>(N, x, y);
  CUDA_CHECK(cudaGetLastError()); // Check for launch errors

  // Wait for GPU to finish before accessing on host
  CUDA_CHECK(cudaDeviceSynchronize());

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  CUDA_CHECK(cudaFree(x));
  CUDA_CHECK(cudaFree(y));

  return 0;
}

Writing check_printf2.cu


In [14]:
%%shell

nvcc -arch=sm_75 check_printf2.cu -o check_printf2
nvprof ./check_printf2

==18107== NVPROF is profiling process 18107, command: ./check_printf2
Thread (0, 0): blockIdx=(0, 0), threadIdx=(0, 0), index=0, n=1048576, stride=1048576, blockDim=(16, 16), gridDim=(64, 64)
Thread (1, 0): blockIdx=(0, 0), threadIdx=(1, 0), index=1, n=1048576, stride=1048576, blockDim=(16, 16), gridDim=(64, 64)
Thread (2, 0): blockIdx=(0, 0), threadIdx=(2, 0), index=2, n=1048576, stride=1048576, blockDim=(16, 16), gridDim=(64, 64)
Thread (3, 0): blockIdx=(0, 0), threadIdx=(3, 0), index=3, n=1048576, stride=1048576, blockDim=(16, 16), gridDim=(64, 64)
Thread (4, 0): blockIdx=(0, 0), threadIdx=(4, 0), index=4, n=1048576, stride=1048576, blockDim=(16, 16), gridDim=(64, 64)
Thread (5, 0): blockIdx=(0, 0), threadIdx=(5, 0), index=5, n=1048576, stride=1048576, blockDim=(16, 16), gridDim=(64, 64)
Thread (6, 0): blockIdx=(0, 0), threadIdx=(6, 0), index=6, n=1048576, stride=1048576, blockDim=(16, 16), gridDim=(64, 64)
Thread (7, 0): blockIdx=(0, 0), threadIdx=(7, 0), index=7, n=1048576, stride

## Where to From Here

If you enjoyed this notebook and want to learn more, the [NVIDIA DLI](https://nvidia.com/dli) offers several in depth CUDA Programming courses.

For those of you just starting out, please consider [_Fundamentals of Accelerated Computing with CUDA C/C++_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-01+V1/about) which provides dedicated GPU resources, a more sophisticated programming environment, use of the [NVIDIA Nsight Systems™](https://developer.nvidia.com/nsight-systems) visual profiler, dozens of interactive exercises, detailed presentations, over 8 hours of material, and the ability to earn a DLI Certificate of Competency.

Similarly, for Python programmers, please consider [_Fundamentals of Accelerated Computing with CUDA Python_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-02+V1/about).

For more intermediate and advance CUDA programming materials, please check out the _Accelerated Computing_ section of the NVIDIA DLI [self-paced catalog](https://www.nvidia.com/en-us/training/online/).